In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine, text

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

format_dict = {'amt':'{:,.2f}','net':'{:,.2f}','refund':'{:,.2f}',
              'grs_amt':'{:,.2f}','net_amt':'{:,.2f}','number':'{:,}',
               'ppu':'{:,.4f}'}
year = 2024
pay_date = date.today()
pay_date

datetime.date(2025, 1, 2)

In [3]:
pay_date = date(2024, 12, 30)
pay_date

datetime.date(2024, 12, 30)

In [5]:
sql = '''
SELECT *
FROM dividends
LIMIT 1'''
print(sql)
df = pd.read_sql(sql, conpf)
df.dtypes


SELECT *
FROM dividends
LIMIT 1


id            int64
stock_id      int64
name         object
year          int64
quarter       int64
number        int64
ppu         float64
amt         float64
net         float64
x_date       object
p_date       object
dtype: object

### Restart and run all

In [8]:
sql = '''
SELECT name, p_date, amt, net, amt-net AS refund, year, quarter AS q
FROM dividends
WHERE YEAR(p_date) = %s AND p_date <= '%s'
ORDER BY p_date DESC, name ASC'''
sql = sql % (year, pay_date)
print(sql)

dtl_by_month = pd.read_sql(sql, conpf)
dtl_by_month.style.format(format_dict)


SELECT name, p_date, amt, net, amt-net AS refund, year, quarter AS q
FROM dividends
WHERE YEAR(p_date) = 2024 AND p_date <= '2024-12-30'
ORDER BY p_date DESC, name ASC


,name,p_date,amt,net,refund,year,q
0,TFFIF,2024-12-27,"2,384.00","2,384.00",0.00,2024,3
1,GVREIT,2024-12-25,"11,988.00","10,789.20","1,198.80",2024,3
2,WHAIR,2024-12-17,"11,145.00","10,030.50","1,114.50",2024,3
3,AIMIRT,2024-12-13,"2,687.50","2,418.75",268.75,2024,3
4,WHART,2024-12-13,"3,860.00","3,474.00",386.00,2024,3
5,CPNREIT,2024-12-12,"12,171.50","10,954.35","1,217.15",2024,3
6,ORI,2024-12-12,75.60,945.00,-869.40,2024,3
7,PTG,2024-12-12,360.00,324.00,36.00,2024,3
8,IVL,2024-12-11,"1,260.00","1,134.00",126.00,2024,3
9,DIF,2024-12-06,"7,777.00","6,999.30",777.70,2024,3


In [10]:
sql = '''
SELECT name, p_date, number, ppu, amt, net
FROM dividends
WHERE YEAR(p_date) = %s AND p_date <= '%s'
ORDER BY p_date, name ASC'''
sql = sql % (year, pay_date)
print(sql)

out_by_month = pd.read_sql(sql, conpf)
out_by_month.style.format(format_dict)


SELECT name, p_date, number, ppu, amt, net
FROM dividends
WHERE YEAR(p_date) = 2024 AND p_date <= '2024-12-30'
ORDER BY p_date, name ASC


,name,p_date,number,ppu,amt,net
0,DIF,2024-03-04,"30,000",0.2264,"6,792.00","6,112.80"
1,GVREIT,2024-03-13,"66,000",0.2044,"13,490.40","12,141.36"
2,CPNREIT,2024-03-15,"60,000",0.2570,"15,420.00","13,878.00"
3,TFFIF,2024-03-20,"20,000",0.1039,"2,078.00","2,078.00"
4,WHART,2024-03-22,"20,000",0.1300,"2,600.00","2,340.00"
5,AIMIRT,2024-03-28,"10,000",0.2150,"2,150.00","1,935.00"
6,WHAIR,2024-03-29,"50,000",0.1369,"6,845.00","6,160.50"
7,SCC,2024-04-23,600,3.5000,"2,100.00","1,890.00"
8,PTTGC,2024-04-24,"6,000",0.7500,"4,500.00","4,050.00"
9,ASK,2024-04-30,"4,500",1.1600,"5,220.00","4,698.00"


In [12]:
dtl_by_month['p_date'] = pd.to_datetime(dtl_by_month['p_date'])
dtl_by_month.dtypes

name              object
p_date    datetime64[ns]
amt              float64
net              float64
refund           float64
year               int64
q                  int64
dtype: object

### Input dividend to YTD Profit

In [15]:
sql = '''
SELECT YEAR(p_date) AS pay_year, SUM(amt) AS grs_amt, SUM(net) AS net_amt, SUM(amt-net) AS refund
FROM dividends
WHERE p_date <= '%s'
GROUP BY pay_year
ORDER BY pay_year DESC'''
sql = sql % pay_date
print(sql)
ttl_by_year = pd.read_sql(sql, conpf)
ttl_by_year.style.format(format_dict)


SELECT YEAR(p_date) AS pay_year, SUM(amt) AS grs_amt, SUM(net) AS net_amt, SUM(amt-net) AS refund
FROM dividends
WHERE p_date <= '2024-12-30'
GROUP BY pay_year
ORDER BY pay_year DESC


,pay_year,grs_amt,net_amt,refund
0,2024,"391,694.05","360,795.14","30,898.91"
1,2023,"520,124.25","481,979.57","38,144.68"
2,2022,"708,367.91","661,087.04","47,280.87"
3,2021,"752,315.10","700,874.50","51,440.60"
4,2020,"616,228.94","576,030.84","40,198.10"
5,2019,"552,424.34","510,266.46","42,157.88"
6,2018,"351,895.60","309,725.58","42,170.02"
7,2017,"245,981.10","219,514.23","26,466.87"


### This one is by year, quarter to compare by year

In [18]:
sql = '''
SELECT year, SUM(amt) AS grs_amt, SUM(net) AS net_amt, SUM(amt-net) AS refund
FROM dividends
GROUP BY year
ORDER BY year DESC'''
print(sql)
ttl_by_year = pd.read_sql(sql, conpf)
ttl_by_year.style.format(format_dict)


SELECT year, SUM(amt) AS grs_amt, SUM(net) AS net_amt, SUM(amt-net) AS refund
FROM dividends
GROUP BY year
ORDER BY year DESC


,year,grs_amt,net_amt,refund
0,2024,"192,591.65","176,707.84","15,883.81"
1,2023,"443,011.65","411,178.45","31,833.20"
2,2022,"646,064.60","602,271.43","43,793.17"
3,2021,"758,673.21","710,977.48","47,695.73"
4,2020,"603,486.53","563,137.46","40,349.07"
5,2019,"601,462.95","557,915.29","43,547.66"
6,2018,"605,859.60","539,821.18","66,038.42"
7,2017,"102,343.50","93,374.15","8,969.35"
8,2016,"185,537.60","164,890.08","20,647.52"
